In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `C:\Users\nikla\Archiv\Master\Sem3\AMO\Project`
Precompiling project...
    883.3 ms  ✓ HiGHS_jll
┌ Warning: attempting to remove probably stale pidfile
│   path = "C:\\Users\\nikla\\.julia\\compiled\\v1.11\\HiGHS\\WFic5_Ul3dR.ji.pidfile"
└ @ FileWatching.Pidfile C:\Users\nikla\.julia\juliaup\julia-1.11.2+0.x64.w64.mingw32\share\julia\stdlib\v1.11\FileWatching\src\pidfile.jl:249
  12399.8 ms  ✓ HiGHS
  2 dependencies successfully precompiled in 15 seconds. 55 already precompiled.


In [4]:
import Pkg; Pkg.add("DataFrames")

   Resolving package versions...
   Installed Crayons ───────────────────── v4.1.1
   Installed TableTraits ───────────────── v1.0.1
   Installed InlineStrings ─────────────── v1.4.2
   Installed SentinelArrays ────────────── v1.4.8
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed PooledArrays ──────────────── v1.4.3
   Installed Tables ────────────────────── v1.12.0
   Installed PrettyTables ──────────────── v2.4.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed LaTeXStrings ──────────────── v1.4.0
   Installed InvertedIndices ───────────── v1.3.1
   Installed StringManipulation ────────── v0.4.0
   Installed DataFrames ────────────────── v1.7.0
    Updating `C:\Users\nikla\Archiv\Master\Sem3\AMO\Project\Project.toml`
  [a93c6f00] + DataFrames v1.7.0
    Updating `C:\Users\nikla\Archiv\Master\Sem3\AMO\Project\Manifest.toml`
  [a8cc5b0e] + Crayons v4.1.1
  [9a962f9c] + DataAPI v1.16.0
  [a93c6f00] + DataFrames v1.7.0
  [e2d170a0] + DataValueInterfaces v1.0.

In [5]:
using HiGHS
using JuMP
using DataFrames

In [18]:
wind_scenario = Dict(1 => 0.8, 2 => 0.85, 3 => 0.9, 4 => 0.75, 5 => 0.6, 6 => 0.5, 
                     7 => 0.55, 8 => 0.6, 9 => 0.65, 10 => 0.7, 11 => 0.75, 12 => 0.8, 
                     13 => 0.85, 14 => 0.9, 15 => 0.95, 16 => 1.0, 17 => 0.9, 18 => 0.8, 
                     19 => 0.7, 20 => 0.6, 21 => 0.5, 22 => 0.4, 23 => 0.3, 24 => 0.2)

Dict{Int64, Float64} with 24 entries:
  5  => 0.6
  16 => 1.0
  20 => 0.6
  12 => 0.8
  24 => 0.2
  8  => 0.6
  17 => 0.9
  1  => 0.8
  19 => 0.7
  22 => 0.4
  23 => 0.3
  6  => 0.5
  11 => 0.75
  9  => 0.65
  14 => 0.9
  3  => 0.9
  7  => 0.55
  4  => 0.75
  13 => 0.85
  15 => 0.95
  2  => 0.85
  10 => 0.7
  18 => 0.8
  21 => 0.5

In [44]:
generators = Dict(
    :G1 => Dict(
            :capacity => 100.0,
            :offer => 12.0,
            ),
    :G2 => Dict(                 # Wind turbine
            :capacity => 80.0,
            :offer => 0
            )
    )

Dict{Symbol, Dict{Symbol}} with 2 entries:
  :G2 => Dict{Symbol, Real}(:offer=>0, :capacity=>80.0)
  :G1 => Dict(:offer=>12.0, :capacity=>100.0)

In [45]:
loads = Dict(
    :D1 => Dict(
            :capacity => 70.0,
            :bid => 40.0,
            ),
    :D2 => Dict(
            :capacity => 30.0,
            :bid => 35.0
            )
    )

Dict{Symbol, Dict{Symbol, Float64}} with 2 entries:
  :D2 => Dict(:bid=>35.0, :capacity=>30.0)
  :D1 => Dict(:bid=>40.0, :capacity=>70.0)

In [46]:
busses = Dict(
    :B1 => Dict(
            :generators => [:G1],
            :loads => [],
            :from => [:L1, :L2],  # from = lines going out from here "give"
            :to => [],            # to = lines going into here "receive"
        ),
    :B2 => Dict(
            :generators => [:G2],
            :loads => [],
            :from => [:L3],
            :to => [:L1]
        ),
    :B3 => Dict(
            :generators => [],
            :loads => [:D1],
            :from => [],
            :to => [:L2],
        ),
    :B4 => Dict(
            :generators => [],
            :loads => [:D2],
            :from => [],
            :to => [:L3],
        )
)

Dict{Symbol, Dict{Symbol, Vector}} with 4 entries:
  :B3 => Dict(:generators=>Any[], :loads=>[:D1], :from=>Any[], :to=>[:L2])
  :B4 => Dict(:generators=>Any[], :loads=>[:D2], :from=>Any[], :to=>[:L3])
  :B1 => Dict(:generators=>[:G1], :loads=>Any[], :from=>[:L1, :L2], :to=>Any[])
  :B2 => Dict(:generators=>[:G2], :loads=>Any[], :from=>[:L3], :to=>[:L1])

In [47]:
lines = Dict(
    :L1 => Dict(
            :from => :B1,
            :to => :B2,
            :capacity => 100.0,
            :susceptance => 500.0
        ),
    :L2 => Dict(
            :from => :B1,
            :to => :B3,
            :capacity => 100.0,
            :susceptance => 500.0
        ),
    :L3 => Dict(
            :from => :B2,
            :to => :B4,
            :capacity => 100.0,
            :susceptance => 500.0
        ),
)

Dict{Symbol, Dict{Symbol, Any}} with 3 entries:
  :L2 => Dict(:from=>:B1, :to=>:B3, :susceptance=>500.0, :capacity=>100.0)
  :L1 => Dict(:from=>:B1, :to=>:B2, :susceptance=>500.0, :capacity=>100.0)
  :L3 => Dict(:from=>:B2, :to=>:B4, :susceptance=>500.0, :capacity=>100.0)

In [48]:

m2 = Model(HiGHS.Optimizer)

# as before
@variable(m2, 0 <= pg[i in keys(generators), t in 1:24] <= generators[i][:capacity])
@variable(m2, 0 <= pd[i in keys(loads), t in 1:24] <= loads[i][:capacity])

    
@objective(m2, Max, 
    sum(pd[i, t] * loads[i][:bid] for i in keys(loads), t in 1:24) - 
    sum(pg[i, t] * generators[i][:offer] for i in keys(generators), t in 1:24)
)

# new variables, expressions and constraints
@variable(m2, δ[b in keys(busses), t in 1:24])
@expression(m2, pf[l in keys(lines), t in 1:24], lines[l][:susceptance] * (δ[lines[l][:from], t] - δ[lines[l][:to], t]))
@constraint(m2, line_cap_pos[l in keys(lines), t in 1:24], pf[l, t] <= lines[l][:capacity])
@constraint(m2, line_cap_neg[l in keys(lines), t in 1:24], pf[l, t] >= -lines[l][:capacity])
@constraint(m2, [t in 1:24], δ[:B1, t] == 0)

@constraint(m2, λ[b in keys(busses), t in 1:24], 
    sum(pg[g, t] for g in busses[b][:generators]) -
    sum(pd[d, t] for d in busses[b][:loads]) - 
    sum(pf[l, t] for l in busses[b][:from]) + 
    sum(pf[l, t] for l in busses[b][:to]) == 0
)

@constraint(m2, [t in 1:24], pg[:G2, t] <= wind_scenario[t] * generators[:G2][:capacity])

print(m2)

In [49]:
optimize!(m2)

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+03]
  Cost   [1e+01, 4e+01]
  Bound  [3e+01, 1e+02]
  RHS    [2e+01, 1e+02]
Presolving model
120 rows, 144 cols, 312 nonzeros  0s
24 rows, 96 cols, 96 nonzeros  0s
14 rows, 56 cols, 56 nonzeros  0s
Presolve : Reductions: rows 14(-274); columns 56(-136); elements 56(-616)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
         14    -7.9488000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 14
Objective value     :  7.9488000000e+04
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00


In [50]:
println("SW = ", objective_value(m2))

# Printing generator outputs over time
for i in keys(generators)
    for t in 1:24
        println(i, " at t=", t, " = ", value(pg[i, t]))
    end
end

# Printing load consumption over time
for i in keys(loads)
    for t in 1:24
        println(i, " at t=", t, " = ", value(pd[i, t]))
    end
end

# Printing shadow prices (dual variables) at each bus for each time slot
for bus in keys(busses)
    for t in 1:24
        println("λ"*string(bus)*" at t=", t, " = ", shadow_price(λ[bus, t]))
    end
end

# Printing power flow on lines over time
for l in keys(lines)
    for t in 1:24
        println("pf on "*string(l)*" at t=", t, " = ", value(pf[l, t]))
    end
end


SW = 79488.0
G2 at t=1 = 64.0
G2 at t=2 = 68.0
G2 at t=3 = 72.0
G2 at t=4 = 60.0
G2 at t=5 = 48.0
G2 at t=6 = 40.0
G2 at t=7 = 44.0
G2 at t=8 = 48.0
G2 at t=9 = 52.0
G2 at t=10 = 56.0
G2 at t=11 = 60.0
G2 at t=12 = 64.0
G2 at t=13 = 68.0
G2 at t=14 = 72.0
G2 at t=15 = 76.0
G2 at t=16 = 80.0
G2 at t=17 = 72.0
G2 at t=18 = 64.0
G2 at t=19 = 56.0
G2 at t=20 = 48.0
G2 at t=21 = 40.0
G2 at t=22 = 32.0
G2 at t=23 = 24.0
G2 at t=24 = 16.0
G1 at t=1 = 36.0
G1 at t=2 = 32.0
G1 at t=3 = 28.0
G1 at t=4 = 40.0
G1 at t=5 = 52.0
G1 at t=6 = 60.0
G1 at t=7 = 56.0
G1 at t=8 = 52.0
G1 at t=9 = 48.0
G1 at t=10 = 44.0
G1 at t=11 = 40.0
G1 at t=12 = 36.0
G1 at t=13 = 32.0
G1 at t=14 = 28.0
G1 at t=15 = 24.0
G1 at t=16 = 20.0
G1 at t=17 = 28.0
G1 at t=18 = 36.0
G1 at t=19 = 44.0
G1 at t=20 = 52.0
G1 at t=21 = 60.0
G1 at t=22 = 68.0
G1 at t=23 = 76.0
G1 at t=24 = 84.0
D2 at t=1 = 30.0
D2 at t=2 = 30.0
D2 at t=3 = 30.0
D2 at t=4 = 30.0
D2 at t=5 = 30.0
D2 at t=6 = 30.0
D2 at t=7 = 30.0
D2 at t=8 = 30.0
D2 at